# Plant Recommender Project

## Cluster Modeling

In [26]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans, DBSCAN, SpectralClustering

#### DBSCAN

Clustering plays an important part in this project - it will provide the basis for the suggestion engine later. To create the best possible clustering model, I will need to have a rough estimate of the number of datapoint clusters. I don't intuitively know the number of clusters necesary since the data has a high degree of dimensionality, so to estimate it I will utilize `DBSCAN` clustering:

In [10]:
# Load in the data and scale it
df = pd.read_csv('../datasets/cleaned-data.csv')
df.dropna(inplace=True)

In [11]:
X = df.drop(columns=['id', 'Scientific_Name_x'])
species = df[['id', 'Scientific_Name_x']]

sc = StandardScaler()

X_sc = sc.fit_transform(X)

In [12]:
X.isnull().sum().any()

False

In [21]:
# Fit a DBSCAN model
db = DBSCAN(eps=10, min_samples=2)
db.fit(X_sc)

DBSCAN(eps=10, min_samples=2)

In [22]:
# Find the number of clusters and look at the silouette score
labels = db.labels_
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_clusters_

68

In [23]:
silhouette_score(X_sc, db.labels_)

0.1460248480136029

This score is not great, but that doesn't particularly matter, as this model was instead meant to give a best estimate on the number of clusters needed to represent the data well. The `n_clusters_` of about 70 found here will be indispensible for testing other clustering models, such as KMeans.

#### KMeans

To see if this silouette score can be improved upon, let's try to use a KMeans clustering model:

In [24]:
km = KMeans(n_clusters=70, random_state=42)
km.fit(X_sc)

KMeans(n_clusters=70, random_state=42)

In [25]:
silhouette_score(X_sc, km.labels_)

0.049270928938044745

This actually performed worse than the `DBSCAN` model, so I will scrap this model in favor of a better one for now.

#### Spectral Clustering

In persuit of a better silhoutte score, I will now try using `SpectralClustering`, another clustering model from `scikitlearn`:

In [27]:
spc = SpectralClustering(n_clusters=70)
spc.fit(X_sc)

/Users/josephhicks/opt/anaconda3/lib/python3.8/site-packages/sklearn/manifold/_spectral_embedding.py:245: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
/Users/josephhicks/opt/anaconda3/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:289: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (70). Possibly due to duplicate points in X.
  est = KMeans(


SpectralClustering(n_clusters=70)

In [30]:
silhouette_score(X_sc, spc.labels_)
spc.get_params

0.31690229242790374

This is by far the best performance I've gotten out of a clustering model thus far. Perhaps this can even be improved upon by tuning the model's hyperparameters:

In [44]:
# Iterate over different combinations of hyperparameters 
def spec_clustering_tuner(data, n_clusters=[6, 10], eigen_solvers=None,
                          gammas=[1],
                          assign_labels=None, n_inits=[10], cores=-1, plot=True):
    """Takes in lists of hyperparameters to tune over a spectral clustering model as well as scaled data.
       Outputs a plot of silhoutte scores against a chosen metric and the best parameters found.
    """
    # Initialize empty params dict and scores list for plotting
    params = {}
    scores = []
    
    # Looping, fitting and testing a new model each iteration
#     for label in assign_labels:
    for gamma in gammas:
        for cluster in n_clusters:
            for num in n_inits:
                model = SpectralClustering(n_clusters=cluster, gamma=gamma, n_init=num,
                                           n_jobs=cores)
                model.fit(data)
                        
                # Save the parameters used alongside the associated score
                score = silhouette_score(data, model.labels_)
                scores.append(score)
                        
                        # Make sure the loop can continue just in case two scores are identical
                if score in params.keys():
                    continue
                        
                params[score] = model.get_params
                        
     # Finding the best score
    best_model_params = sorted(params.items(), reverse=True)[0]
    
    # Plotting
    if plot:
        plt.figure(figsize=(12,10))
        plt.title('Silhouette Score Against Clustering Metrics', fontsize=18)
        sns.scatterplot(n_clusters, scores, alpha=0.75, ci=None)
        plt.show()
        print(best_model_params)
        
    else:
        return best_model_params

In [47]:
best = spec_clustering_tuner(data=X_sc, n_clusters=[5, 20, 70, 90], 
                      gammas=[0.5, 1], n_inits=[5, 20], plot=False)

/Users/josephhicks/opt/anaconda3/lib/python3.8/site-packages/sklearn/manifold/_spectral_embedding.py:245: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
/Users/josephhicks/opt/anaconda3/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:289: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  est = KMeans(
/Users/josephhicks/opt/anaconda3/lib/python3.8/site-packages/sklearn/manifold/_spectral_embedding.py:245: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
/Users/josephhicks/opt/anaconda3/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:289: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  est = KMeans(
/Users/j

In [48]:
best

(0.5184634551759015,
 <bound method BaseEstimator.get_params of SpectralClustering(gamma=0.5, n_clusters=5, n_init=5, n_jobs=-1)>)

Tuning this clustering model has led to an even greater improvement in silhoutte score. This will be the model that I'll use for the recommendation engine.

## Building the Recommendation Engine